In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd
df = pd.read_csv('movies_clean_v3.csv')
# def assign_genre_weight(row):
#     # Vérifier si row est une chaîne de caractères
#     if isinstance(row, str):
#         genres = row.split(',')  # Crée une liste de genres à partir de la chaîne
#         if 'Comédie' in genres:  # Vérifie si "Comédie" est dans la liste des genres
#             return 3
#         elif 'Action' in genres or 'Thriller' in genres:  # Vérifie la présence de "Action" ou "Thriller"
#             return 2
#         else:
#             return 1
#     else:
#         # Retourner un poids par défaut pour les valeurs NaN ou non-chaînes
#         return 0  # ou une autre valeur appropriée selon votre logique d'analyse

# df['genre_weight'] = df['genre'].apply(assign_genre_weight)
# Suppression des colonnes non nécessaires et prétraitement
df = df.drop(['scoring_acteurs', 'acteurs'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=0))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)

mae


141285.53370737075

In [13]:

#Sortie 10/04/2024
new_films = pd.DataFrame({
    'titre': ['NOUS, LES LEROY'],  # Assurez-vous que les valeurs soient dans des listes
    'budget': [None],
    'genre': ['Comédie'],
    'pays': ['France'],
    #'acteurs': [['George Clooney', 'CA', 'NJ']],  # Cela dépend de comment vous avez traité cette caractéristique
    'producteur': ['missing'],
    'realisateur': ['missing'],
    'compositeur': ['missing'],
    'studio': ['Apollo Films'],
    'coeff_studio': [1],
    'date': ['2024-04-10'],
    'season': ['Hiver'],
    'scoring_acteurs&realisateur': [0.5],
    'remake': [None],
    'franchise': [None],
})
# faire des prédictions avec le modèle optimisé
new_y_pred = model.predict(new_films)

# ajouter les prédictions à new_data
new_films['pred_entrees'] = new_y_pred
new_films['pred_entrees_dans_cine_jour'] = (new_y_pred/2000)/7
print(new_films)

             titre budget    genre    pays producteur realisateur compositeur  \
0  NOUS, LES LEROY   None  Comédie  France    missing     missing     missing   

         studio  coeff_studio        date season  scoring_acteurs&realisateur  \
0  Apollo Films             1  2024-04-10  Hiver                          0.5   

  remake franchise  pred_entrees  pred_entrees_dans_cine_jour  
0   None      None     805776.75                    57.555482  
